# *This file allows to generate all plots necessary for figures*

# **Useful packages and functions**

In [1]:
using DifferentialEquations, Plots, Polynomials, LaTeXStrings, ColorSchemes, DelimitedFiles
using Statistics, StatsPlots, Random, ProgressMeter, Printf, LinearAlgebra
include("FYON_2022_STG_kinetics.jl") # Loading of DA kinetics of gating variables
include("FYON_2022_STG_models.jl") # Loading of DA model
include("FYON_2022_STG_utils.jl") # Loading of some utils functions
include("FYON_2022_STG_gs_derivatives.jl") # Loading of X_inf derivatives
include("FYON_2022_STG_DIC.jl") # Loading of the DIC and compensation algorithm
include("FYON_2022_STG_neuromodulation.jl"); # Loading of the neuromodulation cells functions

# **Global variables**

In [2]:
# Definition of simulation time (in ms)
const Tfinal = 5000
const tspan  = (0.0, Tfinal)

# Definition of reversal potential values (in mV) and membrane capacitance
const C     = 1. # Membrane capacitance
const VNa   = 50 # Sodium reversal potential
const VK    = -80 # Potassium reversal potential
const VCa   = 80 # Calcium reversal potential
const VH    = -20 # HCN channels reversal potential
const Vleak = -50 # Leak reversal potential

# Definition of voltage range for the DICs
const Vmin = -60 
const Vmax = 0
const V    = range(Vmin, stop=Vmax, step=0.01)

# Definition of the number of cells in the random set
const ncells = 200;

# **Computing PCA of high frequency bursting neurons and homogeneous scaling using Monte Carlo (Figure 1)**

In [3]:
# Initializing some variables
Na_max = 8000
CaT_max = 12
CaS_max = 50
A_max = 600
KCa_max = 250
Kd_max = 350
H_max = 0.7
leak_max = 0.02
N = 7e4
VV_maxmax = 49.95
VV_maxmin = 49.9
VV_minmax = -69.
VV_minmin = -76.
nb_spikes_target = 5
burstiness_min = 3000.
burstiness_max = 7000.
interburst_f_min = 8.8
interburst_f_max = 9.9
intraburst_f_max = 140.
intraburst_f_min = 70.

g_all_MC = readdlm("./data/g_all_MC.dat");

In [4]:
# Number of ion channels in the STG model
nb_channels = 8

# Means on spiking point cloud
mean_MC = zeros(1, nb_channels)
mean!(mean_MC, g_all_MC)

# Defining scaling factors and limits for the plots
s1 = 1.3
s2 = 1.3

# Normalizing data before applying PCA
g_all_MC_norm = deepcopy(g_all_MC)
for i = 1 : nb_channels
    g_all_MC_norm[:, i] = g_all_MC[:, i] / std(g_all_MC[:, i])
end

# PCA on spiking point cloud
cov_matrix_MC = cov(g_all_MC_norm)
eig_MC = eigen(cov_matrix_MC, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_MC_plot = deepcopy(eig_MC)
for i = 1 : nb_channels
    eig_MC_plot.vectors[i, :] = eig_MC_plot.vectors[i, :] * std(g_all_MC[:, i])
end

In [5]:
total_var = sum(eig_MC.values)
eig_val_decreasing = reverse(eig_MC.values)

p1a  = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
            linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
            xticks=[1, 2, 3, 4, 5, 6, 7, 8], color=:black, markersize=6, linewidth=2, yticks=[0, 4])
ylims!((-0.1, 4))
xlabel!("Component")
ylabel!("Eigenvalue")
display(p1a)
savefig(p1a, "./figures/STG_MC_scree.pdf")

In [6]:
CC1 = scatter_matrix(eig_MC_plot, nb_channels, g_all_MC, mean_MC, Na_max, CaT_max,
                     CaS_max, A_max, KCa_max, Kd_max, H_max, leak_max, s1*1.4, s2*1.4, :gray70, :none)
display(CC1)
savefig(CC1, "./figures/STG_MC_matrix.pdf")

In [7]:
# Computing scaling factors for homogeneous scaling
gNa_scaling = mean_MC[1] / mean_MC[8]
gCaT_scaling = mean_MC[2] / mean_MC[8]
gCaS_scaling = mean_MC[3] / mean_MC[8]
gA_scaling = mean_MC[4] / mean_MC[8]
gKCa_scaling = mean_MC[5] / mean_MC[8]
gKd_scaling = mean_MC[6] / mean_MC[8]
gH_scaling = mean_MC[7] / mean_MC[8]

# Computing alignment
scaling = [gNa_scaling/Na_max, gCaT_scaling/CaT_max, gCaS_scaling/CaS_max, gA_scaling/A_max, 
           gKCa_scaling/KCa_max, gKd_scaling/Kd_max, gH_scaling/H_max, 1/leak_max]
scaling = scaling ./ norm(scaling)

PC1 = eig_MC_plot.vectors[:, nb_channels]
PC1 = [PC1[1]/Na_max, PC1[2]/CaT_max, PC1[3]/CaS_max, PC1[4]/A_max, PC1[5]/KCa_max, 
       PC1[6]/Kd_max, PC1[7]/H_max, PC1[8]/leak_max]
PC1 = PC1 ./ norm(PC1)

alignment = abs(PC1' * scaling)
text = @sprintf "Alignement = %.2f%%" alignment*100
display(text)

"Alignement = 82.17%"

In [8]:
# Computing angles
theta_ACaS_leak = atan((gCaS_scaling/CaS_max)/
                       (gA_scaling/A_max))
theta_ACaS = atan((eig_MC_plot.vectors[:, nb_channels][3]/CaS_max)/
                  (eig_MC_plot.vectors[:, nb_channels][4]/A_max))

theta_CaSH_leak = atan((gH_scaling/H_max)/
                       (gCaS_scaling/CaS_max))
theta_CaSH = atan((eig_MC_plot.vectors[:, nb_channels][7]/H_max)/
                  (eig_MC_plot.vectors[:, nb_channels][3]/CaS_max))

theta_NaKCa_leak = atan((gKCa_scaling/KCa_max)/
                        (gNa_scaling/Na_max))
theta_NaKCa = atan((eig_MC_plot.vectors[:, nb_channels][5]/KCa_max)/
                   (eig_MC_plot.vectors[:, nb_channels][1]/Na_max))

theta_KdA_leak = atan((gA_scaling/A_max)/
                      (gKd_scaling/Kd_max))
theta_KdA = atan((eig_MC_plot.vectors[:, nb_channels][4]/A_max)/
                 (eig_MC_plot.vectors[:, nb_channels][6]/Kd_max));

In [9]:
p1b = plot([theta_ACaS, theta_ACaS], [0, 1], proj=:polar, linewidth=3, label="", yticks=[], 
           yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
           color=1, xtickfontsize=10)
plot!([theta_ACaS_leak, theta_ACaS_leak], [0, 1], proj=:polar, linewidth=3, label="", 
      color=1, linestyle=:dashdotdot)
plot!([theta_CaSH, theta_CaSH], [0, 1], proj=:polar, linewidth=3, label="", 
      color=2)
plot!([theta_CaSH_leak, theta_CaSH_leak], [0, 1], proj=:polar, linewidth=3, label="", 
      color=2, linestyle=:dashdotdot)
plot!([theta_NaKCa, theta_NaKCa], [0, 1], proj=:polar, linewidth=3, label="", 
      color=3)
plot!([theta_NaKCa_leak, theta_NaKCa_leak], [0, 1], proj=:polar, linewidth=3, label="", 
      color=3, linestyle=:dashdotdot)
plot!([theta_KdA, theta_KdA], [0, 1], proj=:polar, linewidth=3, label="", 
      color=4)
plot!([theta_KdA_leak, theta_KdA_leak], [0, 1], proj=:polar, linewidth=3, label="", 
      color=4, linestyle=:dashdotdot)
ylims!(0, 1)
display(p1b)
savefig(p1b, "./figures/STG_MC_polar_leak.pdf")

In [10]:
p1c = scatter(g_all_MC[:, 4], g_all_MC[:, 3], label="", color=:gray70, grid=false,
              yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
              markerstrokewidth=0.)

plot!([mean_MC[4] - s1*2*eig_MC_plot.vectors[:, nb_channels][4]*eig_MC_plot.values[nb_channels], 
      mean_MC[4] + s1*2*eig_MC_plot.vectors[:, nb_channels][4]*eig_MC_plot.values[nb_channels]],
      [mean_MC[3] - s1*2*eig_MC_plot.vectors[:, nb_channels][3]*eig_MC_plot.values[nb_channels], 
      mean_MC[3] + s1*2*eig_MC_plot.vectors[:, nb_channels][3]*eig_MC_plot.values[nb_channels]],
      arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_MC[4] - 0.05*gA_scaling, 
      mean_MC[4] + 0.05*gA_scaling],
      [mean_MC[3] - 0.05*gCaS_scaling, 
      mean_MC[3] + 0.05*gCaS_scaling],
      arrow=false, color=:firebrick1, linewidth=4, label="", linestyle=:dashdotdot)

ylabel!(L"\bar{g}_{CaS} (mS/cm^2)")
xlabel!(L"\bar{g}_A (mS/cm^2)")
ylims!((0, 50))
xlims!((0, 600))
display(p1c)
savefig(p1c, "./figures/neuromod_gCaSA_MC.pdf")

# **Projecting data in the DICs space and computing gs(Vth) isohyperplanes of high frequency bursting neurons using Monte Carlo (Figure 2)**

In [11]:
g_all_Vth = readdlm("./data/g_all_Vth.dat")

ICs_th_Vth = readdlm("./data/ICs_th_Vth.dat");

In [81]:
pNaCaS = scatter(g_all_Vth[:, 1], g_all_Vth[:, 3], label="", grid=false,
                 yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
                 zcolor=ICs_th_Vth[:, 1], c=cgrad(:thermal, rev=false), legend=false,
                 yticks=[0, 12], xticks=[600, 2000], markerstrokewidth=0, tickfontsize=10)

xlabel!(L"\bar{g}_{Na} (mS/cm^2)")
ylabel!(L"\bar{g}_{CaS} (mS/cm^2)")
ylims!((0, 12))
xlims!((600, 2000))

pCaTCaS = scatter(g_all_Vth[:, 2], g_all_Vth[:, 3], label="", grid=false,
                  yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
                  zcolor=ICs_th_Vth[:, 1], c=cgrad(:thermal, rev=false), legend=false,
                  yticks=false, xticks=[0, 6], markerstrokewidth=0, tickfontsize=10)

xlabel!(L"\bar{g}_{CaT} (mS/cm^2)")
ylims!((0, 12))
xlims!((0, 6))

pNaCaT = scatter(g_all_Vth[:, 1], g_all_Vth[:, 2], label="", grid=false,
                 yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
                 zcolor=ICs_th_Vth[:, 1], c=cgrad(:thermal, rev=false), legend=false,
                 yticks=[0, 6], xticks=false, markerstrokewidth=0, tickfontsize=10)

ylabel!(L"\bar{g}_{CaT} (mS/cm^2)")
ylims!((0, 6))
xlims!((600, 2000))

pgfgu = scatter(ICs_th_Vth[:, 3], ICs_th_Vth[:, 6], label="", grid=false,
                yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
                zcolor=ICs_th_Vth[:, 1], c=cgrad(:thermal, rev=false), legend=false,
                xticks=[-10, 2], yticks=false, mirror=true, markerstrokewidth=0, tickfontsize=10)

xlabel!(L"g_f(V_{th})")
ylims!((-2, 6))
xlims!((-10, 2))

pgsgu = scatter(ICs_th_Vth[:, 4], ICs_th_Vth[:, 6], label="", grid=false,
                yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
                zcolor=ICs_th_Vth[:, 1], c=cgrad(:thermal, rev=false), legend=false,
                xticks=[-6, 10], yticks=[-2, 6], mirror=true, markerstrokewidth=0, tickfontsize=10)

xlabel!(L"g_s(V_{th})")
ylabel!(L"g_u(V_{th})")
ylims!((-2, 6))
xlims!((-6, 10))

pgsgf = scatter(ICs_th_Vth[:, 4], ICs_th_Vth[:, 3], label="", grid=false,
                yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
                zcolor=ICs_th_Vth[:, 1], c=cgrad(:thermal, rev=false), legend=false,
                xticks=false, yticks=[-10, 2], mirror=true, markerstrokewidth=0, tickfontsize=10)

ylabel!(L"g_f(V_{th})")
ylims!((-10, 2))
xlims!((-6, 10))


CC2 = plot(pNaCaT, pgfgu, pgsgu, pNaCaS, pCaTCaS, pgsgf, size =(1200, 600),
           layout = @layout([° ° °; ° ° °]), margin=6Plots.mm)
display(CC2)
savefig(CC2, "./figures/STG_DICs_Vth.pdf")

In [13]:
# Extracting threshold voltage and weighting factors for a specific neuron
Random.seed!(543)
(gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak) = g_all_MC[Int(ceil(rand(1, 1)[1]*ncells)), :]
(ith, iosc, gf, gs, gu, gin, Istatic) = DICs(V, 0., gNa, gCaT, gCaS, gA, gKCa, gKd, gH, gleak)
Vth = V[ith]

# Defining the 3 time constants of the 3 timescales
tau_fast = tau_mNa(Vth)
tau_slow = tau_mKd(Vth)
tau_uslow = tau_mH(Vth)

# Computing the wfs(V) and wsu(V) for all gating variables
(wfs_mNa, wsu_mNa) = var_contribution(tau_mNa(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_hNa, wsu_hNa) = var_contribution(tau_hNa(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_mCaT, wsu_mCaT) = var_contribution(tau_mCaT(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_hCaT, wsu_hCaT) = var_contribution(tau_hCaT(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_mCaS, wsu_mCaS) = var_contribution(tau_mCaS(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_hCaS, wsu_hCaS) = var_contribution(tau_hCaS(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_mA, wsu_mA) = var_contribution(tau_mA(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_hA, wsu_hA) = var_contribution(tau_hA(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_mKd, wsu_mKd) = var_contribution(tau_mKd(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_mKCa, wsu_mKCa) = var_contribution(tau_mKCa(Vth), tau_fast, tau_slow, tau_uslow)
(wfs_mH, wsu_mH) = var_contribution(tau_mH(Vth), tau_fast, tau_slow, tau_uslow)

# Modifying scaling factors
s2 = 1.3

# Computing derivatives of gs(Vth) with respect to maximal ion channel conductances
dgs_dNa = (wsu_mNa - wfs_mNa)*3*mNa_inf(Vth)^2*hNa_inf(Vth)*(Vth-VNa)*dmNa(Vth) + 
          (wsu_hNa - wfs_hNa)*mNa_inf(Vth)^3*(Vth-VNa)*dhNa(Vth)
dgs_dCaS = (wsu_mCaS - wfs_mCaS)*3*mCaS_inf(Vth)^2*hCaS_inf(Vth)*(Vth-VCa)*dmCaS(Vth) + 
           (wsu_hCaS - wfs_hCaS)*mCaS_inf(Vth)^3*(Vth-VCa)*dhCaS(Vth)
dgs_dCaT = (wsu_mCaT - wfs_mCaT)*3*mCaT_inf(Vth)^2*hCaT_inf(Vth)*(Vth-VCa)*dmCaT(Vth) + 
           (wsu_hCaT - wfs_hCaT)*mCaT_inf(Vth)^3*(Vth-VCa)*dhCaT(Vth)
dgs_dKd = (wsu_mKd - wfs_mKd)*4*mKd_inf(Vth)^3*(Vth-VK)*dmKd(Vth)
dgs_dA = (wsu_mA - wfs_mA)*3*mA_inf(Vth)^2*hA_inf(Vth)*(Vth-VK)*dmA(Vth) + 
         (wsu_hA - wfs_hA)*mA_inf(Vth)^3*(Vth-VK)*dhA(Vth)
dgs_dH = (wsu_mH - wfs_mH)*(Vth-VH)*dmH(Vth);

In [14]:
theta_ACaS_DIC = -atan((dgs_dA/CaS_max)/(dgs_dCaS/A_max))
theta_ACaS = atan((eig_MC_plot.vectors[:, nb_channels-1][3]/CaS_max)/
                  (eig_MC_plot.vectors[:, nb_channels-1][4]/A_max))
theta_ACaS4 = atan((eig_MC_plot.vectors[:, nb_channels-4][3]/CaS_max)/
                   (eig_MC_plot.vectors[:, nb_channels-4][4]/A_max))

theta_CaTCaS_DIC = -atan((dgs_dCaT/CaS_max)/(dgs_dCaS/CaT_max))
theta_CaTCaS = atan((eig_MC_plot.vectors[:, nb_channels-1][3]/CaS_max)/
                    (eig_MC_plot.vectors[:, nb_channels-1][2]/CaT_max))

theta_NaKd_DIC = -atan((dgs_dNa/Kd_max)/(dgs_dKd/Na_max))
theta_NaKd = atan((eig_MC_plot.vectors[:, nb_channels-1][6]/Kd_max)/
                  (eig_MC_plot.vectors[:, nb_channels-1][1]/Na_max))

theta_CaTA_DIC = -atan((dgs_dCaT/A_max)/(dgs_dA/CaT_max))
theta_CaTA = atan((eig_MC_plot.vectors[:, nb_channels-1][4]/A_max)/
                  (eig_MC_plot.vectors[:, nb_channels-1][2]/CaT_max))
theta_CaTA4 = atan((eig_MC_plot.vectors[:, nb_channels-3][4]/A_max)/
                   (eig_MC_plot.vectors[:, nb_channels-3][2]/CaT_max));

In [15]:
p2a = plot([theta_ACaS, theta_ACaS], [0, 1], proj=:polar, linewidth=3, label="", yticks=[], 
           yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, 
           color=1, xtickfontsize=10, linestyle=:dash)
plot!([theta_ACaS4, theta_ACaS4], [0, 1], proj=:polar, linewidth=2, label="", 
      color=1, linestyle=:dot)
plot!([theta_ACaS_DIC, theta_ACaS_DIC], [0, 1], proj=:polar, linewidth=3, label="", 
      color=1, linestyle=:dashdotdot)
plot!([theta_CaTCaS, theta_CaTCaS], [0, 1], proj=:polar, linewidth=3, label="", 
      color=2, linestyle=:dash)
plot!([theta_CaTCaS_DIC, theta_CaTCaS_DIC], [0, 1], proj=:polar, linewidth=3, label="", 
      color=2, linestyle=:dashdotdot)
plot!([theta_NaKd, theta_NaKd], [0, 1], proj=:polar, linewidth=3, label="", 
      color=3, linestyle=:dash)
plot!([theta_NaKd_DIC, theta_NaKd_DIC], [0, 1], proj=:polar, linewidth=3, label="", 
      color=3, linestyle=:dashdotdot)
plot!([theta_CaTA, theta_CaTA], [0, 1], proj=:polar, linewidth=3, label="", 
      color=4, linestyle=:dash)
plot!([theta_CaTA4, theta_CaTA4], [0, 1], proj=:polar, linewidth=2, label="", 
      color=4, linestyle=:dot)
plot!([theta_CaTA_DIC, theta_CaTA_DIC], [0, 1], proj=:polar, linewidth=3, label="", 
      color=4, linestyle=:dashdotdot)
ylims!(0, 1)
display(p2a)
savefig(p2a, "./figures/STG_MC_polar_DIC.pdf")

In [16]:
p2b = scatter(g_all_MC[:, 4], g_all_MC[:, 3], label="", color=:gray70, grid=false,
              yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
              markerstrokewidth=0.)

plot!([mean_MC[4] - s2*2*eig_MC_plot.vectors[:, nb_channels-1][4]*eig_MC_plot.values[nb_channels-1], 
      mean_MC[4] + s2*2*eig_MC_plot.vectors[:, nb_channels-1][4]*eig_MC_plot.values[nb_channels-1]],
      [mean_MC[3] - s2*2*eig_MC_plot.vectors[:, nb_channels-1][3]*eig_MC_plot.values[nb_channels-1], 
      mean_MC[3] + s2*2*eig_MC_plot.vectors[:, nb_channels-1][3]*eig_MC_plot.values[nb_channels-1]],
      arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

plot!([mean_MC[4] - s2*6*eig_MC_plot.vectors[:, nb_channels-4][4]*eig_MC_plot.values[nb_channels-4], 
      mean_MC[4] + s2*6*eig_MC_plot.vectors[:, nb_channels-4][4]*eig_MC_plot.values[nb_channels-4]],
      [mean_MC[3] - s2*6*eig_MC_plot.vectors[:, nb_channels-4][3]*eig_MC_plot.values[nb_channels-4], 
      mean_MC[3] + s2*6*eig_MC_plot.vectors[:, nb_channels-4][3]*eig_MC_plot.values[nb_channels-4]],
      arrow=false, color=:black, linewidth=2.6, label="", linestyle=:dot)

plot!([mean_MC[4] - 10000000* dgs_dCaS, mean_MC[4] + 10000000*dgs_dCaS],
      [mean_MC[3] + 10000000*dgs_dA, mean_MC[3] - 10000000*dgs_dA],
      arrow=false, color=:firebrick1, linewidth=4, label="", linestyle=:dashdotdot)

ylabel!(L"\bar{g}_{CaS} (mS/cm^2)")
xlabel!(L"\bar{g}_A (mS/cm^2)")
ylims!((0, 50))
xlims!((0, 600))
display(p2b)
savefig(p2b, "./figures/neuromod_gCaSA_MC_DIC.pdf")

# **Computing PCA of high frequency bursting neurons using DICs method (Figure 3)**

In [17]:
g_all_bursting = readdlm("./data/g_all_bursting.dat")
g_all_bursting_DIC = readdlm("./data/g_all_bursting_DIC.dat")
g_all_bursting_leak = readdlm("./data/g_all_bursting_leak.dat")
g_all_bursting_many = readdlm("./data/g_all_bursting_many.dat")

# Definition of the number of cells in the random set
const ncells = 500;

In [18]:
# Number of ion channels in the STG model
nb_channels = 8

# Means on bursting point cloud
mean_bursting = zeros(1, nb_channels)
mean!(mean_bursting, g_all_bursting)

# Defining scaling factors and limits for the plots
s1 = 1.3
s2 = 1.3
Na_max = 8000
CaT_max = 12
CaS_max = 50
A_max = 600
KCa_max = 250
Kd_max = 350
H_max = 0.7
leak_max = 0.02

# Normalizing data before applying PCA
g_all_bursting_norm = deepcopy(g_all_bursting)
for i = 1 : nb_channels
    g_all_bursting_norm[:, i] = g_all_bursting[:, i] / std(g_all_bursting[:, i])
end

# PCA on spiking point cloud
cov_matrix_bursting = cov(g_all_bursting_norm)
eig_bursting = eigen(cov_matrix_bursting, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_bursting_plot = deepcopy(eig_bursting)
for i = 1 : nb_channels
    eig_bursting_plot.vectors[i, :] = eig_bursting_plot.vectors[i, :] * std(g_all_bursting[:, i])
end

In [19]:
p3a = scatter(g_all_bursting[:, 4], g_all_bursting[:, 3], label="", color=:gray70, grid=false,
              yguidefontsize=18, xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
              markerstrokewidth=0.)

plot!([mean_bursting[4] - s1*eig_bursting_plot.vectors[:, nb_channels][4]*eig_bursting_plot.values[nb_channels], 
       mean_bursting[4] + s1*eig_bursting_plot.vectors[:, nb_channels][4]*eig_bursting_plot.values[nb_channels]],
       [mean_bursting[3] - s1*eig_bursting_plot.vectors[:, nb_channels][3]*eig_bursting_plot.values[nb_channels], 
       mean_bursting[3] + s1*eig_bursting_plot.vectors[:, nb_channels][3]*eig_bursting_plot.values[nb_channels]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_bursting[4] - s2*1.5*eig_bursting_plot.vectors[:, nb_channels-1][4]*eig_bursting_plot.values[nb_channels-1], 
       mean_bursting[4] + s2*1.5*eig_bursting_plot.vectors[:, nb_channels-1][4]*eig_bursting_plot.values[nb_channels-1]],
       [mean_bursting[3] - s2*1.5*eig_bursting_plot.vectors[:, nb_channels-1][3]*eig_bursting_plot.values[nb_channels-1], 
       mean_bursting[3] + s2*1.5*eig_bursting_plot.vectors[:, nb_channels-1][3]*eig_bursting_plot.values[nb_channels-1]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

pearson_cor_bursting = cor(g_all_bursting[:, 3], g_all_bursting[:, 4])
str_pearson_cor_bursting = @sprintf "%.2f" pearson_cor_bursting
annotate!(300, 20, Plots.text(L"r =  %$str_pearson_cor_bursting", :black, :center, 12))

ylabel!(L"\bar{g}_{CaS} (mS/cm^2)")
xlabel!(L"\bar{g}_A (mS/cm^2)")
ylims!((0, 50))
xlims!((0, 400))
display(p3a)
savefig(p3a, "./figures/PCA_STG_neuromod_gCaSA_bursting.pdf")

In [20]:
p3b = scatter(g_all_bursting_DIC[:, 4], g_all_bursting_DIC[:, 3], label="", 
              markershape=:cross, color=:gray70, grid=false, yguidefontsize=18, 
              xguidefontsize=18, legendfontsize=12, margin=5Plots.mm, legend=false)
scatter!(g_all_bursting_leak[:, 4], g_all_bursting_leak[:, 3], label="", 
         color=:gray70, markershape=:utriangle, markerstrokewidth=0.)

ylabel!(L"\bar{g}_{CaS} (mS/cm^2)")
xlabel!(L"\bar{g}_A (mS/cm^2)")
ylims!((0, 50))
xlims!((0, 400))
display(p3b)
savefig(p3b, "./figures/neuromod_gCaLCaN_separation_spiking.pdf")

In [21]:
p3c = scatter(g_all_bursting_many[:, 4], g_all_bursting_many[:, 3], label="", 
              zcolor=g_all_bursting_many[:, 8], grid=false, yguidefontsize=18, xguidefontsize=18, 
              legendfontsize=12, margin=5Plots.mm, c=cgrad(:thermal, rev=false), legend=false, 
              markerstrokewidth=0.)

ylabel!(L"\bar{g}_{CaS} (mS/cm^2)")
xlabel!(L"\bar{g}_A (mS/cm^2)")
ylims!((0, 50))
xlims!((0, 400))
display(p3c)
savefig(p3c, "./figures/neuromod_many_gCaLCaN_zgleak_spiking.pdf")

# **Computing PCA of neuromodulated neurons using DICs method (Figure 4)**

In [22]:
g_all_spiking = readdlm("./data/g_all_spiking.dat")
g_all_spiking_DIC = readdlm("./data/g_all_spiking_DIC.dat")
g_all_spiking_leak = readdlm("./data/g_all_spiking_leak.dat")

g_all_doublets = readdlm("./data/g_all_doublets.dat")
g_all_doublets_DIC = readdlm("./data/g_all_doublets_DIC.dat")
g_all_doublets_leak = readdlm("./data/g_all_doublets_leak.dat")

burstiness_doublets = readdlm("./data/burstiness_doublets.dat")
burstiness_doublets_plot = readdlm("./data/burstiness_doublets_plot.dat")
burstiness_doublets_DIC = readdlm("./data/burstiness_doublets_DIC.dat")
burstiness_doublets_DIC_plot = readdlm("./data/burstiness_doublets_DIC_plot.dat")
burstiness_doublets_leak = readdlm("./data/burstiness_doublets_leak.dat")
burstiness_doublets_leak_plot = readdlm("./data/burstiness_doublets_leak_plot.dat")
burstiness_bursting = readdlm("./data/burstiness_bursting.dat")
burstiness_bursting_DIC = readdlm("./data/burstiness_bursting_DIC.dat")
burstiness_bursting_leak = readdlm("./data/burstiness_bursting_leak.dat");

In [23]:
# Defining colors for the color map
mySalmon = RGB{Float64}(243. / 255., 124. / 255., 130. / 255.)
myYellow = RGB{Float64}(253. / 255., 211. / 255., 44. / 255.)
mycmap = ColorScheme([mySalmon, myYellow]);

In [24]:
# Defining the number of conductances
nb_channels = 8

# Defining a matrix containing all conductances
g_all_bar = zeros(ncells, nb_channels*3)
scaling =  [1, 700, 150, 15, 30, 25, 9000, 400000]
g_all_spiking_bar = g_all_spiking ./ 1e3
g_all_doublets_bar = g_all_doublets ./ 1e3
g_all_bursting_bar = g_all_bursting ./ 1e3
for i = 1 : nb_channels
    g_all_bar[:, (i-1)*3 + 1] = g_all_spiking_bar[:, i] .* scaling[i]
    g_all_bar[:, (i-1)*3 + 2] = g_all_doublets_bar[:, i] .* scaling[i]
    g_all_bar[:, (i-1)*3 + 3] = g_all_bursting_bar[:, i] .* scaling[i]
end

p4a = bar([1:4:29], mean(g_all_bar[:, 1:3:22], dims=1)', label="", grid=false, yguidefontsize=18, 
          color=:gray70, ytickfontsize=15, xguidefontsize=15, legendfontsize=12, 
          margin=3Plots.mm, yticks=[0, 6],  xrotation=0, xticks=false, 
          yerror=std(g_all_bar[:, 1:3:22], dims=1)', bar_width=1, ylabel="Conductance (S/cm²)")

bar!([2:4:30], mean(g_all_bar[:, 2:3:23], dims=1)', label="", grid=false, yguidefontsize=18, 
    color=mySalmon, ytickfontsize=15, xguidefontsize=15, legendfontsize=12, 
    margin=3Plots.mm, yticks=[0, 6],  xrotation=0, xticks=false, 
    yerror=std(g_all_bar[:, 2:3:23], dims=1)', bar_width=1)

bar!([3:4:31], mean(g_all_bar[:, 3:3:24], dims=1)', label="", grid=false, yguidefontsize=18, 
    color=myYellow, ytickfontsize=15, xguidefontsize=15, legendfontsize=12, xtickfontsize=25,
    margin=3Plots.mm, yticks=[0, 6],  xrotation=0, yerror=std(g_all_bar[:, 3:3:24], dims=1)', 
    bar_width=1, xticks=(2:4:30, [L"\bar{g}_{Na}", L"\bar{g}_{CaT}", L"\bar{g}_{CaS}", L"\bar{g}_{A}", 
                                  L"\bar{g}_{KCa}", L"\bar{g}_{Kd}", L"\bar{g}_{H}", L"g_{leak}"]))

xlims!((0., 32.))
ylims!((0, 6))

CC4 = plot(p4a, layout=(1, 1), legend=false, size=(807,500, 300))
display(CC4)
savefig(CC4, "./figures/neuromod_gCaSA_gbar.pdf")

In [25]:
# Means on spiking point cloud
mean_spiking = zeros(1, nb_channels)
mean!(mean_spiking, g_all_spiking)

# Means on triplets point cloud
mean_doublets = zeros(1, nb_channels)
mean!(mean_doublets, g_all_doublets)

# Defining scaling factors for the plots
s1 = 1.3
s2 = 2.2

# Normalizing data before applying PCA
g_all_spiking_norm = deepcopy(g_all_spiking)
for i = 1 : nb_channels
    g_all_spiking_norm[:, i] = g_all_spiking[:, i] / std(g_all_spiking[:, i])
end

g_all_doublets_norm = deepcopy(g_all_doublets)
for i = 1 : nb_channels
    g_all_doublets_norm[:, i] = g_all_doublets[:, i] / std(g_all_doublets[:, i])
end

# PCA on triplets point cloud
cov_matrix_spiking = cov(g_all_spiking_norm)
eig_spiking = eigen(cov_matrix_spiking, sortby = x -> abs(x))

# PCA on doublets point cloud
cov_matrix_doublets = cov(g_all_doublets_norm)
eig_doublets = eigen(cov_matrix_doublets, sortby = x -> abs(x))

eig_spiking_plot = deepcopy(eig_spiking)
for i = 1 : nb_channels
    eig_spiking_plot.vectors[i, :] = eig_spiking_plot.vectors[i, :] * std(g_all_spiking[:, i])
end

eig_doublets_plot = deepcopy(eig_doublets)
for i = 1 : nb_channels
    eig_doublets_plot.vectors[i, :] = eig_doublets_plot.vectors[i, :] * std(g_all_doublets[:, i])
end

In [78]:
# Initializing a flag
flag_start = true

# Removing unsuccessful simulations
for i = 1 : ncells
    if !isnan.(burstiness_doublets_plot)[i]
        if flag_start
            # Creating the conductance matrix
            global g_all_doublets_plot = g_all_doublets[i, :]
            flag_start = false
        else
            # Adding a line to the conductance matrix
            g_all_doublets_plot = hcat(g_all_doublets_plot, g_all_doublets[i, :])
        end
    end
end

# Transposing the conductance matrix
if @isdefined g_all_doublets_plot
    g_all_doublets_plot = g_all_doublets_plot'
end;

In [79]:
p4b = scatter(g_all_spiking[:, 4], g_all_spiking[:, 3], label="", grid=false, yguidefontsize=18, xguidefontsize=18, 
              margin=5Plots.mm, legend=false, color=:gray70, colorbar_title="burstiness",
              colorbar_titlefontsize=18, markerstrokewidth=0.)
scatter!(g_all_doublets_plot[:, 4], g_all_doublets_plot[:, 3], label="", 
         zcolor=burstiness_doublets, c=cgrad(mycmap, rev=false), markerstrokewidth=0.)
scatter!(g_all_bursting[:, 4], g_all_bursting[:, 3], label="", 
         zcolor=burstiness_bursting, c=cgrad(mycmap, rev=false),
         markerstrokewidth=0.)

plot!([mean_spiking[4] - s1*eig_spiking_plot.vectors[:, nb_channels][4]*eig_spiking_plot.values[nb_channels], 
       mean_spiking[4] + s1*eig_spiking_plot.vectors[:, nb_channels][4]*eig_spiking_plot.values[nb_channels]],
       [mean_spiking[3] - s1*eig_spiking_plot.vectors[:, nb_channels][3]*eig_spiking_plot.values[nb_channels], 
       mean_spiking[3] + s1*eig_spiking_plot.vectors[:, nb_channels][3]*eig_spiking_plot.values[nb_channels]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_spiking[4] - s2*eig_spiking_plot.vectors[:, nb_channels-1][4]*eig_spiking_plot.values[nb_channels-1], 
       mean_spiking[4] + s2*eig_spiking_plot.vectors[:, nb_channels-1][4]*eig_spiking_plot.values[nb_channels-1]],
       [mean_spiking[3] - s2*eig_spiking_plot.vectors[:, nb_channels-1][3]*eig_spiking_plot.values[nb_channels-1], 
       mean_spiking[3] + s2*eig_spiking_plot.vectors[:, nb_channels-1][3]*eig_spiking_plot.values[nb_channels-1]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

plot!([mean_doublets[4] - s1*eig_doublets_plot.vectors[:, nb_channels][4]*eig_doublets_plot.values[nb_channels], 
       mean_doublets[4] + s1*eig_doublets_plot.vectors[:, nb_channels][4]*eig_doublets_plot.values[nb_channels]],
       [mean_doublets[3] - s1*eig_doublets_plot.vectors[:, nb_channels][3]*eig_doublets_plot.values[nb_channels], 
       mean_doublets[3] + s1*eig_doublets_plot.vectors[:, nb_channels][3]*eig_doublets_plot.values[nb_channels]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_doublets[4] - s2*eig_doublets_plot.vectors[:, nb_channels-1][4]*eig_doublets_plot.values[nb_channels-1], 
       mean_doublets[4] + s2*eig_doublets_plot.vectors[:, nb_channels-1][4]*eig_doublets_plot.values[nb_channels-1]],
       [mean_doublets[3] - s2*eig_doublets_plot.vectors[:, nb_channels-1][3]*eig_doublets_plot.values[nb_channels-1], 
       mean_doublets[3] + s2*eig_doublets_plot.vectors[:, nb_channels-1][3]*eig_doublets_plot.values[nb_channels-1]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

plot!([mean_bursting[4] - s1*eig_bursting_plot.vectors[:, nb_channels][4]*eig_bursting_plot.values[nb_channels], 
       mean_bursting[4] + s1*eig_bursting_plot.vectors[:, nb_channels][4]*eig_bursting_plot.values[nb_channels]],
       [mean_bursting[3] - s1*eig_bursting_plot.vectors[:, nb_channels][3]*eig_bursting_plot.values[nb_channels], 
       mean_bursting[3] + s1*eig_bursting_plot.vectors[:, nb_channels][3]*eig_bursting_plot.values[nb_channels]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:solid)

plot!([mean_bursting[4] - s2*eig_bursting_plot.vectors[:, nb_channels-1][4]*eig_bursting_plot.values[nb_channels-1], 
       mean_bursting[4] + s2*eig_bursting_plot.vectors[:, nb_channels-1][4]*eig_bursting_plot.values[nb_channels-1]],
       [mean_bursting[3] - s2*eig_bursting_plot.vectors[:, nb_channels-1][3]*eig_bursting_plot.values[nb_channels-1], 
       mean_bursting[3] + s2*eig_bursting_plot.vectors[:, nb_channels-1][3]*eig_bursting_plot.values[nb_channels-1]],
       arrow=false, color=:black, linewidth=4, label="", linestyle=:dash)

pearson_cor_spiking = cor(g_all_spiking[:, 3], g_all_spiking[:, 4])
str_pearson_cor_spiking = @sprintf "%.2f" pearson_cor_spiking
annotate!(500, 10, Plots.text(L"r =  %$str_pearson_cor_spiking", :black, :center, 12))

pearson_cor_doublets = cor(g_all_doublets[:, 3], g_all_doublets[:, 4])
str_pearson_cor_doublets = @sprintf "%.2f" pearson_cor_doublets
annotate!(440, 37, Plots.text(L"r =  %$str_pearson_cor_doublets", :black, :center, 12))

pearson_cor_bursting = cor(g_all_bursting[:, 3], g_all_bursting[:, 4])
str_pearson_cor_bursting = @sprintf "%.2f" pearson_cor_bursting
annotate!(100, 40, Plots.text(L"r =  %$str_pearson_cor_bursting", :black, :center, 12))

ylabel!(L"\bar{g}_{CaS} (mS/cm^2)")
xlabel!(L"\bar{g}_A (mS/cm^2)")
ylims!((0, 50))
xlims!((0, 600))
display(p4b)
savefig(p4b, "./figures/PCA_STG_neuromod_gCaSA_burstiness.pdf")

In [28]:
# Initializing a flag
flag_start = true

# Removing unsuccessful simulations
for i = 1 : ncells
    if !isnan.(burstiness_doublets_DIC_plot)[i]
        if flag_start
            # Creating the conductance matrix
            global g_all_doublets_DIC_plot = g_all_doublets_DIC[i, :]
            flag_start = false
        else
            # Adding a line to the conductance matrix
            g_all_doublets_DIC_plot = hcat(g_all_doublets_DIC_plot, g_all_doublets_DIC[i, :])
        end
    end
end

# Transposing the conductance matrix
if @isdefined g_all_doublets_DIC_plot
    g_all_doublets_DIC_plot = g_all_doublets_DIC_plot'
end;

In [29]:
p4c = scatter(g_all_spiking_DIC[:, 4], g_all_spiking_DIC[:, 3], label="", legend=false,
              markershape=:cross, color=:gray70, grid=false, yguidefontsize=18, 
              xguidefontsize=18, legendfontsize=12, margin=5Plots.mm,
              colorbar_title="burstiness", colorbar_titlefontsize=18, markerstrokewidth=0.)
scatter!(g_all_spiking_leak[:, 4], g_all_spiking_leak[:, 3], label="", 
         color=:gray70, markershape=:utriangle, markerstrokewidth=0.)
scatter!(g_all_doublets_DIC_plot[:, 4], g_all_doublets_DIC_plot[:, 3], label="", markershape=:cross,
         zcolor=burstiness_doublets_DIC, c=cgrad(mycmap, rev=false), markerstrokewidth=0.)
scatter!(g_all_doublets_leak[:, 4], g_all_doublets_leak[:, 3], label="", markerstrokewidth=0.,
         markershape=:utriangle, zcolor=burstiness_doublets_leak, c=cgrad(mycmap, rev=false))
scatter!(g_all_bursting_DIC[:, 4], g_all_bursting_DIC[:, 3], label="", markerstrokewidth=0.,
         markershape=:cross, zcolor=burstiness_bursting_DIC, c=cgrad(mycmap, rev=false))
scatter!(g_all_bursting_leak[:, 4], g_all_bursting_leak[:, 3], label="", markerstrokewidth=0.,
         markershape=:utriangle, zcolor=burstiness_bursting_leak, c=cgrad(mycmap, rev=false))

ylabel!(L"\bar{g}_{CaS} (mS/cm^2)")
xlabel!(L"\bar{g}_A (mS/cm^2)")
ylims!((0, 50))
xlims!((0, 600))
display(p4c)
savefig(p4c, "./figures/neuromod_gCaSA_separation_burstiness.pdf")

# **Plotting neuromodulation paths using DICs method (Figure 5)**

In [30]:
burstiness = readdlm("./data/burstiness_paths.dat")
g_all_init = readdlm("./data/g_all_init_paths.dat")
gCaSs = readdlm("./data/gCaSs_paths.dat")
gAs = readdlm("./data/gAs_paths.dat");

In [31]:
# Initializing some variables
n = size(gCaSs)[2]

# Plotting the results
p5a = plot(legend=false)

# Looping over all neurons
for j = 1 : n
    # Plotting first spiking neurons
    i_thresh = minimum(findall(burstiness[:, j] .> 0))
    if i_thresh > 1
        plot!(gAs[1:i_thresh, j], gCaSs[1:i_thresh, j], label="", grid=false, linewidth=5, 
              yguidefontsize=18, xguidefontsize=18, margin=5Plots.mm, color=:gray70)
    end
    
    # Then plotting bursting neurons
    plot!(gAs[i_thresh:end, j], gCaSs[i_thresh:end, j], label="", 
          line_z=burstiness[i_thresh:end, j], grid=false, linewidth=5, clim=(700, 7000),
          yguidefontsize=18, xguidefontsize=18, margin=5Plots.mm, c=cgrad(mycmap, rev=false))
end
ylabel!(L"\bar{g}_{CaS} (mS/cm^2)")
xlabel!(L"\bar{g}_A (mS/cm^2)")
ylims!((0, 50))
xlims!((0, 600))
display(p5a)
savefig(p5a, "./figures/neuromod_gCaSA_paths.pdf")

In [32]:
# Definition of simulation time (in ms)
const Tfinal = 2015
const tspan  = (0.0, Tfinal)

# Fixing random seed
Random.seed!(2020)

# Retrieving maximal conductances from what had been computed
j = Int(ceil(rand(1, 1)[1] * n))

include("FYON_2022_STG_models.jl") # Loading of STG model

# Computing the regression line along the neuromodulation path
line_ACaS = fit(gAs[:, j], gCaSs[:, j], 1)
a0 = line_ACaS[0]
a1 = line_ACaS[1]
gA_min = minimum(gAs[:, j])
gA_max = maximum(gAs[:, j])

# Extracting the maximal ion channel conductances
(gNa, gCaT, _, _, gKCa, gKd, gH, gleak) = g_all_init[j, :]

# Parameter vector for simulations
p = [0., gNa, gCaT, gA_min, gA_max, gKCa, gKd, gH, gleak, C, a0, a1]

# Initial conditions
V0  = -70.
Ca0 = 0.5
x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
      hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]

# Simulation
prob = ODEProblem(STG_ODE_neuromod, x0, tspan, p)
sol = solve(prob, verbose=false)

# Removing transient part
V_t = sol[1, :][1090:end]
t = sol.t[1090:end]
p5b = plot(t[1:2110], V_t[1:2110], linewidth=1.5, grid=false, xticks=[], yticks=[], color=:gray70, 
           yguidefontsize=18, xguidefontsize=18, legend=false, axis=false)
plot!(t[2110:end], V_t[2110:end], linewidth=1.5, grid=false, xticks=[], yticks=[], 
      line_z=range(0, 7000, length(t[2110:end])), c=cgrad(mycmap, rev=false), yguidefontsize=18, 
      xguidefontsize=18, legend=false, axis=false)
display(p5b)
savefig(p5b, "./figures/neuromod_gCaSA_path_simu1.pdf")

In [33]:
# Definition of simulation time (in ms)
const Tfinal = 2050
const tspan  = (0.0, Tfinal)

# Fixing random seed
Random.seed!(410)

# Retrieving maximal conductances from what had been computed
j = Int(ceil(rand(1, 1)[1] * n))

include("FYON_2022_STG_models.jl") # Loading of STG model

# Computing the regression line along the neuromodulation path
line_ACaS = fit(gAs[:, j], gCaSs[:, j], 1)
a0 = line_ACaS[0]
a1 = line_ACaS[1]
gA_min = minimum(gAs[:, j])
gA_max = maximum(gAs[:, j])

# Extracting the maximal ion channel conductances
(gNa, gCaT, _, _, gKCa, gKd, gH, gleak) = g_all_init[j, :]

# Parameter vector for simulations
p = [0., gNa, gCaT, gA_min, gA_max, gKCa, gKd, gH, gleak, C, a0, a1]

# Initial conditions
V0  = -70.
Ca0 = 0.5
x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
      hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]

# Simulation
prob = ODEProblem(STG_ODE_neuromod, x0, tspan, p)
sol = solve(prob, verbose=false)

# Removing transient part
V_t = sol[1, :][1130:end]
t = sol.t[1130:end]
p5c = plot(t[1:1700], V_t[1:1700], linewidth=1.5, grid=false, xticks=[], yticks=[], color=:gray70, 
           yguidefontsize=18, xguidefontsize=18, legend=false, axis=false)
plot!(t[1700:end], V_t[1700:end], linewidth=1.5, grid=false, xticks=[], yticks=[], 
      line_z=range(0, 7000, length(t[1700:end])), c=cgrad(mycmap, rev=false), yguidefontsize=18, 
      xguidefontsize=18, legend=false, axis=false)
display(p5c)
savefig(p5c, "./figures/neuromod_gCaSA_path_simu2.pdf")

In [34]:
# Definition of simulation time (in ms)
const Tfinal = 2040
const tspan  = (0.0, Tfinal)

# Fixing random seed
Random.seed!(813)

# Retrieving maximal conductances from what had been computed
j = Int(ceil(rand(1, 1)[1] * n))

include("FYON_2022_STG_models.jl") # Loading of STG model

# Computing the regression line along the neuromodulation path
line_ACaS = fit(gAs[:, j], gCaSs[:, j], 1)
a0 = line_ACaS[0]
a1 = line_ACaS[1]
gA_min = minimum(gAs[:, j])
gA_max = maximum(gAs[:, j])

# Extracting the maximal ion channel conductances
(gNa, gCaT, _, _, gKCa, gKd, gH, gleak) = g_all_init[j, :]

# Parameter vector for simulations
p = [0., gNa, gCaT, gA_min, gA_max, gKCa, gKd, gH, gleak, C, a0, a1]

# Initial conditions
V0  = -70.
Ca0 = 0.5
x0  = [V0, mNa_inf(V0), hNa_inf(V0), mCaT_inf(V0), hCaT_inf(V0), mCaS_inf(V0), 
      hCaS_inf(V0), mA_inf(V0), hA_inf(V0), mKCa_inf(V0, Ca0), mKd_inf(V0), mH_inf(V0), Ca0]

# Simulation
prob = ODEProblem(STG_ODE_neuromod, x0, tspan, p)
sol = solve(prob, verbose=false)

# Removing transient part
V_t = sol[1, :][1090:end]
t = sol.t[1090:end]
p5d = plot(t[1:1640], V_t[1:1640], linewidth=1.5, grid=false, xticks=[], yticks=[], color=:gray70, 
           yguidefontsize=18, xguidefontsize=18, legend=false, axis=false)
plot!(t[1640:end], V_t[1640:end], linewidth=1.5, grid=false, xticks=[], yticks=[], 
      line_z=range(0, 7000, length(t[1640:end])), c=cgrad(mycmap, rev=false), yguidefontsize=18, 
      xguidefontsize=18, legend=false, axis=false)
display(p5d)
savefig(p5d, "./figures/neuromod_gCaSA_path_simu3.pdf")

# **Computing PCA on homogeneous scaled neurons using Monte Carlo (Figure supp 1)**

In [35]:
g_all_MC_leak = readdlm("./data/g_all_MC_leak.dat")
f_leak = readdlm("./data/f_leak.dat")
Iapp = readdlm("./data/Iapp.dat")
rheobase = readdlm("./data/rheobase.dat")

# Definition of the number of cells in the random set
const ncells = 200;

In [36]:
# Number of ion channels in the STG model
nb_channels = 8

# Means on spiking point cloud
mean_MC_leak = zeros(1, nb_channels)
mean!(mean_MC_leak, g_all_MC_leak)

# Defining scaling factors and limits for the plots
s1 = 1.3
s2 = 1.3
Na_max = 8000
CaT_max = 12
CaS_max = 50
A_max = 600
KCa_max = 250
Kd_max = 350
H_max = 0.7
leak_max = 0.02

# Normalizing data before applying PCA
g_all_MC_leak_norm = deepcopy(g_all_MC_leak)
for i = 1 : nb_channels
    g_all_MC_leak_norm[:, i] = g_all_MC_leak[:, i] / std(g_all_MC_leak[:, i])
end

# PCA on spiking point cloud
cov_matrix_MC_leak = cov(g_all_MC_leak_norm)
eig_MC_leak = eigen(cov_matrix_MC_leak, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_MC_leak_plot = deepcopy(eig_MC_leak)
for i = 1 : nb_channels
    eig_MC_leak_plot.vectors[i, :] = eig_MC_leak_plot.vectors[i, :] * std(g_all_MC_leak[:, i])
end

In [37]:
total_var = sum(eig_MC_leak.values)
eig_val_decreasing = reverse(eig_MC_leak.values)
ps1a = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
           linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
           xticks=[1, 2, 3, 4, 5, 6, 7, 8], color=:black, markersize=6, linewidth=2, yticks=[0, 9])
ylims!((-0.2, 9))
xlabel!("Component")
ylabel!("Eigenvalue")
display(ps1a)
savefig(ps1a, "./figures/STG_MC_scree_leak.pdf")

In [38]:
CCs1 = scatter_matrix_zcolor(eig_MC_leak_plot, nb_channels, g_all_MC_leak, mean_MC, Na_max*1.1, CaT_max,
                             CaS_max, A_max, KCa_max, Kd_max, H_max, leak_max, s1*2, 0, rheobase, :none)
display(CCs1)
savefig(CCs1, "./figures/STG_MC_matrix_leak.pdf")

In [39]:
# Initializing some variables
f_leak_plot = deepcopy(f_leak)

# Plotting the I f curves
ps1b = plot()
for i = 1 : ncells
    # Removing local overshoots when releasing
    index = findall(f_leak_plot[:, i] .> 40.)
    for ii in index
        f_leak_plot[ii, i] = (f_leak_plot[ii-1, i] + f_leak_plot[ii+1, i])/2
    end
    
    plot!(Iapp, f_leak_plot[:, i], label="", grid=false, yguidefontsize=15, xguidefontsize=15, 
          legendfontsize=12, margin=5Plots.mm, color=:black, alpha=0.1, linewidth=2, 
          xticks=[-8, 0], yticks=[0, 40])
end
xlabel!(L"I (mA)")
ylabel!(L"f (Hz)")
xlims!((-8, 0))
ylims!((0, 40))
display(ps1b)
savefig(ps1b, "./figures/simu_STG_MC_hyper_leak.pdf")

# **Plotting high frequency bursting neurons generated using the DICs method (Figure supp 2)**

In [40]:
total_var = sum(eig_bursting.values)
eig_val_decreasing = reverse(eig_bursting.values)

ps2 = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
           linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
           xticks=[1, 2, 3, 4, 5, 6, 7, 8], color=:black, markersize=6, linewidth=2, yticks=[0, 6])
ylims!((-0.1, 6))
xlabel!("Component")
ylabel!("Eigenvalue")
display(ps2)
savefig(ps2, "./figures/STG_PCA_bursting_scree.pdf")

In [41]:
CCs2 = scatter_matrix(eig_bursting_plot, nb_channels, g_all_bursting, mean_bursting, Na_max, CaT_max,
                     CaS_max, A_max, KCa_max, Kd_max, H_max, leak_max, s1, s2*2, :gray70, :none)
display(CCs2)
savefig(CCs2, "./figures/STG_PCA_bursting_matrix.pdf")

# **Computing PCA on high frequency bursting neurons generated using the DICs method, property separated (Figure supp 3&4)**

In [42]:
# Definition of the number of cells in the random set
const ncells = 500;

In [43]:
# Means on bursting point cloud
mean_bursting_leak = zeros(1, nb_channels)
mean!(mean_bursting_leak, g_all_bursting_leak)

# Defining scaling factors and limits for the plots
s1 = 1.3
s2 = 1.3
Na_max = 8000
CaT_max = 12
CaS_max = 50
A_max = 600
KCa_max = 250
Kd_max = 350
H_max = 0.7
leak_max = 0.02

# Normalizing data before applying PCA
g_all_bursting_leak_norm = deepcopy(g_all_bursting_leak)
for i = 1 : nb_channels
    g_all_bursting_leak_norm[:, i] = g_all_bursting_leak[:, i] / std(g_all_bursting_leak[:, i])
end

# PCA on spiking point cloud
cov_matrix_bursting_leak = cov(g_all_bursting_leak_norm)
eig_bursting_leak = eigen(cov_matrix_bursting_leak, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_bursting_leak_plot = deepcopy(eig_bursting_leak)
for i = 1 : nb_channels
    eig_bursting_leak_plot.vectors[i, :] = eig_bursting_leak_plot.vectors[i, :] * std(g_all_bursting_leak[:, i])
end

In [44]:
total_var = sum(eig_bursting_leak.values)
eig_val_decreasing = reverse(eig_bursting_leak.values)
ps3 = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
           linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
           xticks=[1, 2, 3, 4, 5, 6, 7, 8], color=:black, markersize=6, linewidth=2, yticks=[0, 9])
ylims!((-0.2, 9))
xlabel!("Component")
ylabel!("Eigenvalue")
display(ps3)
savefig(ps3, "./figures/STG_PCA_bursting_scree_leak.pdf")

In [45]:
CCs3 = scatter_matrix(eig_bursting_leak_plot, nb_channels, g_all_bursting_leak, mean_bursting_leak, 
                      Na_max*1.1, CaT_max, CaS_max, A_max, KCa_max, Kd_max, H_max, leak_max, s1, 0, 
                      :gray70, :utriangle)
display(CCs3)
savefig(CCs3, "./figures/STG_PCA_bursting_leak_matrix.pdf")

In [46]:
# Means on bursting point cloud
mean_bursting_DIC = zeros(1, nb_channels)
mean!(mean_bursting_DIC, g_all_bursting_DIC)

# Defining scaling factors and limits for the plots
s1 = 1.5
s2 = 1.5
Na_max = 8000
CaT_max = 12
CaS_max = 50
A_max = 600
KCa_max = 250
Kd_max = 350
H_max = 0.7
leak_max = 0.02

# Normalizing data before applying PCA
g_all_bursting_DIC_norm = deepcopy(g_all_bursting_DIC)
for i = 1 : nb_channels-1
    g_all_bursting_DIC_norm[:, i] = g_all_bursting_DIC[:, i] / std(g_all_bursting_DIC[:, i])
end

# PCA on spiking point cloud
cov_matrix_bursting_DIC = cov(g_all_bursting_DIC_norm)
eig_bursting_DIC = eigen(cov_matrix_bursting_DIC, sortby = x -> abs(x))

# Computing PCA components to map back to the non-normalized dataset
eig_bursting_DIC_plot = deepcopy(eig_bursting_DIC)
for i = 1 : nb_channels
    eig_bursting_DIC_plot.vectors[i, :] = eig_bursting_DIC_plot.vectors[i, :] * std(g_all_bursting_DIC[:, i])
end

In [47]:
total_var = sum(eig_bursting_DIC.values)
eig_val_decreasing = reverse(eig_bursting_DIC.values)
ps4 = plot(eig_val_decreasing, label="", grid=false, yguidefontsize=15, markershape=:diamond, 
           linestyle=:dashdot, xguidefontsize=15, legendfontsize=12, margin=5Plots.mm, 
           xticks=[1, 2, 3, 4, 5, 6, 7, 8], color=:black, markersize=6, linewidth=2, yticks=[0, 5])
ylims!((-0.1, 5))
xlabel!("Component")
ylabel!("Eigenvalue")
display(ps4)
savefig(ps4, "./figures/STG_PCA_bursting_scree_DIC.pdf")

In [48]:
eig_bursting_DIC_plot.vectors[:, nb_channels-1] = eig_bursting_DIC_plot.vectors[:, nb_channels]
eig_bursting_DIC_plot.values[nb_channels-1]=eig_bursting_DIC_plot.values[nb_channels]
CCs4 = scatter_matrix(eig_bursting_DIC_plot, nb_channels, g_all_bursting_DIC, 
                      mean_bursting_DIC, Na_max, CaT_max, CaS_max, A_max, KCa_max, Kd_max, 
                      H_max, leak_max, 0, s2, :gray70, :cross)
display(CCs4)
savefig(CCs4, "./figures/STG_PCA_bursting_DIC_matrix.pdf")